In [3]:
import requests
import json
import pandas as pd


GRAPHQL_BASE_URI = 'https://ancient-oasis-74228.herokuapp.com/graphql'
#req_all_transactions = { "query" : "{ viewer  { login } }" }

def get_all_transactions():
    req_all_transactions = {"query": "query { allTransactions {transactionId accountId transactionDay category transactionAmount}}"}
    r = requests.post(url=GRAPHQL_BASE_URI, json=req_all_transactions)
    return r.json()

data = get_all_transactions()

In [21]:
transactions = data["data"]["allTransactions"]
df = pd.DataFrame(transactions)

df['category'].unique()
df.groupby('accountId').head()

,accountId,category,transactionAmount,transactionDay,transactionId
0,A22,BB,285.05,2,Z00014
1,A22,BB,285.00,2,Z00013
2,A22,BB,285.00,2,Z00012
3,A20,AA,707.00,29,T000991
4,A30,EE,874.00,29,T000990
5,A35,FF,945.00,2,T00099
6,A6,GG,543.00,29,T000989
7,A19,DD,912.00,29,T000988
8,A33,GG,410.00,29,T000987
9,A7,FF,363.00,29,T000986


In [107]:
def splitDataFrameList(df, target_column):
    row_accumulator = []
    def splitListToRows(row):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1)
    new_df = pd.DataFrame(row_accumulator)
    return new_df

'a,b,v'

In [23]:
def get_all_customers():
    req_all_customers = {"query": "query {allCustomers {customerId accounts}}"}

    r = requests.post(url=GRAPHQL_BASE_URI, json=req_all_customers)
    return r.json()

data_customers = get_all_customers()

In [109]:
df_cust = pd.DataFrame(data_customers['data']['allCustomers'])
df_cust.head()

df_cust_split = splitDataFrameList(df_cust, target_column='accounts')
df_cust_split.head()



,accounts,customerId
0,A43,IND0037
1,A92,IND0092
2,A91,IND0092
3,A4,IND0092
4,A10,IND0092


In [56]:
df_concat = df.join(df_cust_split.set_index('accounts'), on='accountId')
df_concat.head(50)
#df_data = df_concat[~df_concat['customerId'].isnull()]
df_data['day_of_week'] = df_data['transactionDay'] % 7
df_data['month'] = (df_data['transactionDay'] % 30) % 12

df_data.head()

,accountId,category,transactionAmount,transactionDay,transactionId,customerId,day_of_week,month
0,A22,BB,285.05,2,Z00014,IND0015,2,2
1,A22,BB,285.00,2,Z00013,IND0015,2,2
2,A22,BB,285.00,2,Z00012,IND0015,2,2
3,A20,AA,707.00,29,T000991,IND0013,1,5
4,A30,EE,874.00,29,T000990,IND0024,1,5


In [94]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

df_data.index = range(0, df_data.shape[0])

le = LabelEncoder()
ohe = OneHotEncoder()

model = le.fit(df_data['category'])
df_data['category_encoded'] = le.transform(df_data['category'])


X = ohe.fit_transform(df_data['category_encoded'].values.reshape(-1,1)).toarray()
category_cols = ["category_"+str(int(i)) for i in range(X.shape[1])]
df_ohe = pd.DataFrame(X, columns = category_cols)
df_data_encoded = pd.concat([df_data, df_ohe], axis=1)



cols_selected = ['day_of_week', 'month']
cols_selected.extend(category_cols)
df_data_encoded.head()
#len(df_ohe.index.unique()), len(df_data.index.unique()) 
#df_ohe.head(), df_data.head()
#df_ohe.index
print(cols_selected)

X = df_data_encoded[cols_selected]
y = df_data_encoded['transactionAmount']


['day_of_week', 'month', 'category_0', 'category_1', 'category_2', 'category_3', 'category_4', 'category_5', 'category_6']


In [117]:
from  sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
clf = rf.fit(X,y)

#clf.predict()


clf.predict(X.iloc[[0],:])

X.head()

,day_of_week,month,category_0,category_1,category_2,category_3,category_4,category_5,category_6
0,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [116]:
TMP_DIR = '/var/tmp/'
GENERIC_MODEL_FILE = TMP_DIR + 'generic.pickle'
import pickle

pickle.dump(clf, open(GENERIC_MODEL_FILE, 'wb'))

try:
    clf2 = pickle.load(open(GENERIC_MODEL_FILE+'kk', 'rb'))
    clf2.predict(X.iloc[[0],:])
except IOError:
   print ("Error: can\'t find file or read data")
    
    

Error: can't find file or read data


In [144]:
import json
jstring = b'[{"transaction": {"transactionDay":800, "category": "AA"}}, {"transaction": {"transactionDay":850, "category": "GG"}}]'

content = json.loads(jstring.decode('UTF-8'))
content_aslist = [x['transaction'] for x in content]
df = pd.DataFrame(content_aslist)


df['day_of_week'] = df['transactionDay'] % 7
df['month'] = (df['transactionDay'] % 30) % 12


df['category_encoded'] = le.transform(df['category'])


X = ohe.transform(df['category_encoded'].values.reshape(-1,1)).toarray()
category_cols = ["category_"+str(int(i)) for i in range(X.shape[1])]
df_ohe = pd.DataFrame(X, columns = category_cols)
df_encoded = pd.concat([df, df_ohe], axis=1)

df_encoded.head()

cols_selected = ['day_of_week', 'month']
cols_selected.extend(category_cols)
X = df_encoded[cols_selected]
X.head()
pred = clf.predict(X)
list(pred)
json.dumps(list(pred))

'[496.1634365634366, 289.3313095238096]'